In [1]:
import sys
import numpy as np
import pandas as pd
import pickle
import os
import pybullet as p

os.environ['KMP_DUPLICATE_LIB_OK']='True'
sys.path.append(os.path.join(os.path.abspath(os.getcwd()),".."))
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

In [2]:
# import FERL Modules
from utils.learned_feature import LearnedFeature
from utils.plot_utils import *
from utils.environment_utils import *

In [3]:
# Settings for the different cases
traces_file_cases = ["laptop", "table", "proxemics", "human"]
traces_idx = np.arange(10).tolist()
resources_dir = parent_dir + "/data/resources/"

# Setup environment.
physicsClient = p.connect(p.DIRECT)
p.setAdditionalSearchPath(resources_dir)
objectID = setup_environment()
p.setGravity(0, 0, 0)
p.setRealTimeSimulation(1)

# settings for the learned feature (27D Euclidean Feature Space)
LF_dict = {'bet_data':5, 'sin':False, 'cos':False, 'rpy':False, 'lowdim':False, 'norot':True,
           'noangles':True, '6D_laptop':False, '6D_human':False, '9D_coffee':False, 'EErot':False,
           'noxyz':False, 'subspace_heuristic':False}

# Learn Laptop Feature from collected feature traces

In [7]:
# Setting for which Case (see paper)
case = 4

In [8]:
# Step 0: Visualize feature to be learned
plot_gt3D(parent_dir, traces_file_cases[case-1], objectID)

In [6]:
# Step 1: load feature traces & initialize a learnable feature
unknown_feature = LearnedFeature(2, 64, LF_dict)

data_file = parent_dir + '/data/user_data/{}.p'.format(traces_file_cases[case-1])
trajectory_list = pickle.load(open( data_file, "rb" ) )

all_trace_data = np.empty((0, 97), float)
for idx in traces_idx:
    np.flip(trajectory_list[idx],axis=0)
    unknown_feature.add_data(trajectory_list[idx])
    all_trace_data = np.vstack((all_trace_data, trajectory_list[idx]))

IndexError: list index out of range

In [ ]:
# 1.1 Visualize the Traces labeled at random with the initialized Network
plot_learned_traj(unknown_feature.function, all_trace_data, objectID)

In [ ]:
# Step 2: train the feature on the set of traces
_ = unknown_feature.train(epochs=100, batch_size=32, learning_rate=1e-3, weight_decay=0.001, s_g_weight=10.)

In [ ]:
# Step 3: Analyze the learned Feature

In [ ]:
# 3.1 Visualize the labeled Traces
plot_learned_traj(unknown_feature.function, all_trace_data, objectID)

In [ ]:
# 3.2 Visualize the learned function over the 3D Reachable Set
plot_learned3D(parent_dir, unknown_feature.function, objectID)